In [1]:
!pip install -q streamlit
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 657.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 3.3 MB/s eta 0:00:00


In [2]:
!ngrok authtoken 2frxdgyN6DxbWSFuxxsjeA87rCe_7ke2X5W8vJNVq6LvEo9u1

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
# Install the SQLAlchemy library if it is not installed
!sudo apt-get install python3-dev libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
!sudo pip3 install -U sql_magic > /dev/null
!pip install psycopg2-binary > /dev/null
# Google colab upgraded the default SQL Alchemy … this break pandas read_sql
# We are downgrading SQL Alchemy for now.
!pip install -U 'sqlalchemy<2.0'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.40
    Uninstalling SQLAlchemy-2.0.40:
      Successfully uninstalled SQLAlchemy-2.0.40
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.54 which is incompatible.


In [4]:
!streamlit run app.py &>/dev/null &

In [5]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(8501)
public_url

ERROR:pyngrok.process.ngrok:t=2025-05-16T18:12:14+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The account \"Krystal Wu\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-16T18:12:14+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The account \"Krystal Wu\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-16T18:12:14+0000 lvl=eror msg="terminating with error" obj=app err="authentication failed: The account \"Krystal Wu\" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The account "Krystal Wu" may not start an ngrok agent session until the admin's email address is verified. Verify your email at https://dashboard.ngrok.com/user/settings\r\n\r\nERR_NGROK_123\r\n.

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from cachetools import cached
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math
import random
import requests


from sqlalchemy import create_engine

conn_string = 'mysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user='WeatherToWear',
    password='6u36kMD9akk=',
    host='jsedocc7.scrc.nyu.edu',
    port=3306,
    db='WeatherToWear',
    encoding = 'utf-8'
)

engine = create_engine(conn_string)
inventory_df=pd.read_sql('SELECT * FROM inventory', engine)

# Rename categories for better visualization
category_counts = inventory_df['category'].value_counts()
category_counts_relabel = category_counts.rename({
    'dresses': 'Dress',
    'top': 'Top',
    'bottom': 'Bottom',
    'knit': 'Knit',
    'jacket_coat': 'Jacket / Coat',
    'jumpsuit_romper': 'Jumpsuit / Romper'
})

PAGE_CONFIG = {"page_title":"WeatherToWear","page_icon":":dress:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

def fetch_weather_details():
    ip_address = requests.get('https://api.ipify.org').text
    ipstack_key = "f8f531469638c9f7215c4fd5626f295f"
    ipstack_url = f'http://api.ipstack.com/{ip_address}?access_key={ipstack_key}'
    ipstack_resp = requests.get(ipstack_url)
    ipstack_data = ipstack_resp.json()

    lat = ipstack_data['latitude']
    lon = ipstack_data['longitude']
    city = ipstack_data['city']

    weather_key = "DDw8RAMKYKhS5LNXIjNsEpNLAyRwuRNQ"
    weather_url = f'https://api.tomorrow.io/v4/weather/realtime?location={lat},{lon}&apikey={weather_key}'
    headers = {'accept': 'application/json'}
    weather_resp = requests.get(weather_url, headers=headers)
    weather_data = weather_resp.json()

    country=ipstack_data['country_name']
    temperature=weather_data['data']['values']['temperature']
    feels_like_temperature=weather_data['data']['values']['temperatureApparent']
    precipitation=weather_data['data']['values']['precipitationProbability']
    humidity=weather_data['data']['values']['humidity']
    uv_index=weather_data['data']['values']['uvIndex']
    wind_gust=weather_data['data']['values']['windGust']

    return {
        'City': city,
        'Country': country,
        'Temperature': temperature,
        'Feels Like Temperature': feels_like_temperature,
        'Precipitation Probability': precipitation,
        'Humidity': humidity,
        'UV Index': uv_index,
        'Wind Gust': wind_gust
    }

def recommend_clothing(weather_data, temp_preference):
    city = weather_data['City']
    temperature = weather_data['Temperature']
    feels_like_temperature = weather_data['Feels Like Temperature']
    precipitation_probability = weather_data['Precipitation Probability']
    humidity = weather_data['Humidity']
    uv_index = weather_data['UV Index']
    wind_gust = weather_data['Wind Gust']

    suitable_conditions = []

    # rain, wind, sun
    if precipitation_probability > 50:
        suitable_conditions.append("waterproof")
    if wind_gust:
        suitable_conditions.extend(["windproof"])
    if uv_index > 5:
        suitable_conditions.extend(["lightweight"])
    if humidity:
        suitable_conditions.extend(["breathable"])

    # temperature
    if temp_preference == 'Warmer':
        if feels_like_temperature < 14:
            suitable_conditions.extend(["insulated"])
        if feels_like_temperature > 25:
            suitable_conditions.extend(["lightweight"])
        if feels_like_temperature > 27:
            suitable_conditions.append("sleeveless")
    elif temp_preference == 'Colder':
        if feels_like_temperature < 8:
            suitable_conditions.extend(["insulated"])
        if feels_like_temperature > 19:
            suitable_conditions.extend(["lightweight"])
        if feels_like_temperature > 22:
            suitable_conditions.append("sleeveless")
    else:  # this is the default yay
        if feels_like_temperature < 10:
            suitable_conditions.extend(["insulated"])
        if feels_like_temperature > 21:
            suitable_conditions.extend(["lightweight"])
        if feels_like_temperature > 24:
            suitable_conditions.append("sleeveless")

    # print recommendations user can choose to take
    st.write(f"Based on the weather in your location and your preferences, we recommend clothing that is:",", ".join(suitable_conditions))


def main():
    menu = ["Welcome","Try it Out", "Our Data"]
    choice = st.sidebar.selectbox('Menu',menu)

    # styling with CSS
    custom_css = """
        <style>
            @import url('"https://fonts.googleapis.com/css2?family=Quicksand:wght@300..700&display=swap"');

            body {
                background-image: url('https://i.pinimg.com/564x/6a/b2/be/6ab2be50feffba39e9088478f03dc12b.jpg');
                background-size: cover;
                background-repeat: no-repeat
                background-attachment: fixed;
                background-color: rgba(255, 255, 255, 0.3);
                font-family: 'Quicksand', sans-serif;
            }
            .stApp {
                margin-top: 100px;
                margin-bottom: 100px;
                margin-right: 120px;
                margin-left: 120px;
                border-radius: 40px; /* Adding border-radius for rounded edges */
                overflow: hidden;
            }
            .stMarkdown {
                font-size: 14px;
                line-height: 1.6;
            }
            .image-collage {
                display: flex;
                flex-wrap: wrap;
            }
            .image-collage img {
                width: 200px;
                height: 200px;
                object-fit: cover;
                margin: 10px;
            }
            h1, h2, h3, h4, h5, h6 {
            font-family: 'Quicksand', sans-serif;
            }
        </style>
    """

    # render with custom css
    st.markdown(custom_css, unsafe_allow_html=True)

    if choice == 'Welcome':
        st.title("Welcome to WeatherToWear")
        image_urls = ["https://cdn.rtrcdn.com/assets/imgs/04192024-SpringHub-Update-Party-714x1071.jpg",
                      "https://cdn.rtrcdn.com/assets/imgs/04192024-SpringHub-Update-Workwear-714x1071.jpg",
                      "https://cdn.rtrcdn.com/assets/imgs/04192024-SpringHub-Update-Wedding-714x1071.jpg",
                      ]

        # Display images in a flexible collage
        st.markdown('<div class="image-collage">{}</div>'.format(''.join('<img src="{}">'.format(url) for url in image_urls)),
                    unsafe_allow_html=True)
        st.info(
          """
          WeatherToWear revolutionizes the daily decision-making process of dressing by seamlessly blending real-time weather intelligence with personal fashion preferences. Don't know whether to wear it or not? We've got you covered!
          """,
          icon="🌷",
        )
        st.success(
          """
          This data product was developed by Brynna Shale, Kristina Sisiakova, and Krystal Wu.
          """,
          icon="🌐",
        )
    elif choice == 'Try it Out':
        # this page should (optionally) collect user's location and have a multiple choice for warmer/colder and style preferences
        st.title("Try it Out")

        # option to integrate weather and temperature
        temp_preference = st.radio("Do you typically like to feel warmer or colder?", ['Warmer', 'Colder'])
        if st.button("Fetch Weather"):
          weather_data = fetch_weather_details()
          st.write("Current weather:", weather_data['Temperature'], "°C, but it feels like",weather_data['Feels Like Temperature'])
          st.write("The UV Index is",weather_data['UV Index'],", and the wind gust is",weather_data['Wind Gust'],".")
          st.write("Chance of precipitation is",weather_data['Precipitation Probability'],"and humidity is",weather_data['Humidity'],".")
          st.write("")
          recommend_clothing(weather_data, temp_preference)

        weather_categories = inventory_df['weather'].unique().tolist()
        categories = inventory_df['category'].unique().tolist()
        styles = inventory_df['style'].unique().tolist()
        colors = inventory_df['color'].unique().tolist()

        st.markdown("**Select a combination of features and see whether we have your dream item in our database!**")
        selected_weather = st.multiselect("Weather Suitability", weather_categories)
        selected_category = st.selectbox("Category", [""] + categories)
        selected_style = st.selectbox("Style", [""] + styles)
        selected_color = st.selectbox("Color", [""] + colors)

        # based on user selection, filter data
        filtered_data = inventory_df.copy()
        if selected_category:
            filtered_data = filtered_data[filtered_data['category'] == selected_category]
        if selected_style:
            filtered_data = filtered_data[filtered_data['style'] == selected_style]
        if selected_color:
            filtered_data = filtered_data[filtered_data['color'] == selected_color]
        if selected_weather:
            filtered_data = filtered_data[filtered_data['weather'].isin(selected_weather)]

        # display a random image based off criteria
        st.markdown("**Here is an example:**")
        if len(filtered_data) > 0:
            random_index = random.randint(0, len(filtered_data) - 1)
            image_url = filtered_data.iloc[random_index]['image']
            st.image(f"https://pc-ap.rtrcdn.com/{image_url}", caption='Image Preview', width=200)
        else:
            st.write("Sorry, we don't have a clothing item matching your selection.")
    elif choice == 'Our Data':
      # shows static and interactive visualizations (?)

      st.title("Dataset Analysis")
      st.info("Our dataset contains clothing items from RentTheRunway website, which features clothes for various occasions and events.")

      st.subheader("Peek into the Dataset:")
      st.write("The dataset includes the clothing images, as well as details about their styles, colors, and weather suitability.")
      st.write("Here are the first 3 rows of the dataset:")
      st.write(inventory_df.head(3))

      st.markdown("**Number of variables:**")
      st.write(f"{inventory_df.shape[1]}")
      st.markdown("**Number of observations:**")
      st.write(f"{inventory_df.shape[0]}")
      st.markdown("**Number of missing observations:**")
      st.dataframe(inventory_df.isnull().sum().to_frame().T)

      st.subheader("Distribution of Clothing Categories:")
      st.write("This bar chart illustrates the distribution of clothing items across different categories, such as dresses, tops, bottoms, etc.")
      st.bar_chart(category_counts_relabel)

      st.subheader("Distribution of Clothing Styles:")
      st.bar_chart(inventory_df['style'].value_counts())

      st.subheader("Distribution of Clothing Weather Suitability:")
      st.bar_chart(inventory_df['weather'].value_counts())

      st.subheader("Distribution of Clothing Colors:")
      st.bar_chart(inventory_df['color'].value_counts())

      st.subheader("Relationships Between the Variables:")
      heatmap_data = inventory_df.groupby(['style', 'weather']).size().unstack(fill_value=0)
      fig, ax = plt.subplots(figsize=(12, 8))
      sns.heatmap(heatmap_data, cmap='coolwarm', annot=True, fmt='d', ax=ax)
      ax.set_title('Style vs Weather Suitability Heatmap')
      ax.set_xlabel('Weather Suitability')
      ax.set_ylabel('Style')
      st.pyplot(fig)

      heatmap_data = inventory_df.groupby(['style', 'color']).size().unstack(fill_value=0)
      fig, ax = plt.subplots(figsize=(12, 8))
      sns.heatmap(heatmap_data, cmap='coolwarm', annot=True, fmt='d', ax=ax)
      ax.set_title('Style vs Color Heatmap')
      ax.set_xlabel('Color')
      ax.set_ylabel('Style')
      st.pyplot(fig)


if __name__ == '__main__':
    main()